# Translate eisenscript to eisenxml using lark

In [1]:
from pathlib import Path
from lark import Lark
from lark.visitors import Interpreter, Transformer, v_args
parser = Lark.open('eisenscript.lark')

In eisen tree structure I have nodes for
 - main
 - global_md
 - entry
 - call
 - loop
 - trans 
 - w_mod
 - md_mod

 will need an Interpreter method for each which adds something to the xml tree  
 how do I get token values?
 look   @visit_children_decor

In [2]:
# minimal example
test = """ 2 * { x 1 y 2} seed
{ rx 1 ry 2} seed
rule seed  { 
  box 
}
"""

In [3]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""

In [4]:
small = parser.parse(test)
small

Tree(Token('RULE', 'main'), [Tree(Token('RULE', 'entry'), [Tree(Token('RULE', 'call'), [Tree('repeat', [Token('NUMBER', '2')]), Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'trans'), [Token('TID', 'x'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'y'), Token('VALUE', '2')])]), Tree('rulename', [Token('RULENAME', 'seed')])]), Tree(Token('RULE', 'call'), [Tree(Token('RULE', 'loop'), [Tree(Token('RULE', 'trans'), [Token('TID', 'rx'), Token('VALUE', '1')]), Tree(Token('RULE', 'trans'), [Token('TID', 'ry'), Token('VALUE', '2')])]), Tree('rulename', [Token('RULENAME', 'seed')])])]), Tree(Token('RULE', 'ruledef'), [Tree('rulename', [Token('RULENAME', 'seed')]), Tree(Token('RULE', 'call'), [Tree('rulename', [Token('RULENAME', 'box')])])])])

In [5]:
print(small.pretty())

main
  entry
    call
      repeat	2
      loop
        trans
          x
          1
        trans
          y
          2
      rulename	seed
    call
      loop
        trans
          rx
          1
        trans
          ry
          2
      rulename	seed
  ruledef
    rulename	seed
    call
      rulename	box



In [6]:
from xml.etree import ElementTree as et
from xml.dom import minidom


In [7]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = et.tostring(elem, "utf-8")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

class Itest(Interpreter):
    
    def convert(self, tree):
        self.xmltree = et.Element('rules')
        self.active = self.xmltree
        self._visit_tree(tree)
        return prettify(self.xmltree)
    
    def main(self, tree):
        self.visit_children(tree)

    def entry(self, tree):
        self.active = et.SubElement(self.xmltree, "rule")
        self.active.set("name", "entry")
        self.visit_children(tree)
        self.active = self.xmltree
        
    def call(self, tree):   
        # the tag is set in rulename method depending if call to rule or instance of shape
        call = et.SubElement(self.active, "dummy tag")
        current = self.active
        self.active = call
        print('len(call kids)', len(tree.children))
        self.visit_children(tree)
        self.active = current

    @v_args(meta=True)
    def trans(self, kids, meta):
        tid = meta[0]
        values = meta[1:]
        
        if tid == "s" and len(values) == 1:
            tid = "sa"
        if tid in ("x", "y", "z"):
            tid = "t" + tid
        if tid in ('tx', 'ty', 'tz', 'rx', 'ry', 'rz', 'sa', 's'):
            tstr = tid + " " + " ".join(values) 
        else:
            tstr = ''
        return tstr


    def loop(self, tree):
        # the list of transforms in {} brackets
        tstr = ' '.join(self.visit_children(tree))
        print(tstr)
        self.active.set("transforms", tstr.strip())

    def ruledef(self, tree):
        self.active = et.SubElement(self.xmltree, "rule")
        self.visit_children(tree)
        self.active = self.xmltree

            
    @v_args(inline=True)
    def rulename(self, rname):
        # can get here from ruledef or call or mmod
        
        if self.active.tag == "dummy tag":
            # parent is call
            if rname in ["box", "grid", "sphere", "line"]:
                self.active.tag = "instance"
                self.active.set("shape", rname)
            else: 
                self.active.tag = "call"
                self.active.set("rule", rname)
        else:
            # parent is ruledef
            self.active.set("name", rname)
        

    @v_args(inline=True)
    def repeat(self, c):
        self.active.set("count", c)

    @v_args(inline=True)
    def wmod(self, weight):
        self.active.set("weight", weight)

    @v_args(meta=True)
    def mmod(self, kids, meta):
        self.active.set("max_depth", meta[0])
        if len(meta) > 1:
            self.active.set("successor", meta[1])
      

In [8]:
it = Itest().convert(small)
print(it)

len(call kids) 3
tx 1 ty 2
len(call kids) 2
rx 1 ry 2
len(call kids) 1
<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call count="2" transforms="tx 1 ty 2" rule="seed"/>
    <call transforms="rx 1 ry 2" rule="seed"/>
  </rule>
  <rule name="seed">
    <instance shape="box"/>
  </rule>
</rules>



Ok we have the the right xml structure for a simple case, now need to get the actual values for rulename, transform string, count attributes into the correct places.

main -> entry -> call -> loop - > trans  
     -> ruledef -> call -> loop -> trans 

transforms - DONE  
count  - DONE  
rulename - DONE  

maybe do need a different grammar for a rule call and a shape instance?  
Or could retag rule from rulename method? - DONE

I think this works!

then rule mods md_mod, w_mod and sucessor rule  - DONE  

FIX grammar to include all set statements but only action set maxdepth
ON principle, grammar should define everything in file, but Interprter only processes what's needed for eisenxml. This would make repurposing grammar parser for some other type of output possible.

FIX multiple (nested) loops per call

In [9]:
ball = """set maxdepth 2000
R1 

rule R1 w 10 md 20  { 
{ x 1  rz 3 ry 5  } R1
{ s 1 1 0.1 sat 0.9 } box
} 
"""
balltree = parser.parse(ball)
print(Itest().convert(balltree))

len(call kids) 1
len(call kids) 2
tx 1 rz 3 ry 5
len(call kids) 2
s 1 1 0.1 
<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call rule="R1"/>
  </rule>
  <rule name="R1" weight="10" max_depth="20">
    <call transforms="tx 1 rz 3 ry 5" rule="R1"/>
    <instance transforms="s 1 1 0.1" shape="box"/>
  </rule>
</rules>



can we make colour a generic string  
"red" 
           | "black"
           | "white"
           | "blue"
           | "green"

oct below has nested rules and currently fails to parse
change   
`call: count* loop* rn`  
to  
`call: (count* loop*)* rn`  

This only gets the last count * loop.  

My old code did this:

```xml
<?xml version="1.0" ?>
<rules max_depth="100">
  <rule name="entry">
    <call transforms="ry 36" count="10" rule="r1_00"/>
  </rule>
  <rule name="r1_00">
    <call transforms="ry 10" count="30" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <instance transforms="s 1 0.2 0.5" shape="box"/>
    <call transforms="sa 0.9 rz 5 rx 5 tx 1" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <instance transforms="s 1 0.2 0.5" shape="box"/>
    <call transforms="sa 0.99 rz -5 rx -5 tx 1" rule="r1"/>
  </rule>
</rules>
```

It's created an extra rule "r1_00" and dropped the third nested rule. The code does say it only deals with two loops per call.

Lark version visits all the loops but keeps overriding the atrributes
Creating the extra rules here could be done by adding to the tree?

Logic:  
if more than one loop per call:
  

item? - Zero or one instances of item (”maybe”)
item* - Zero or more instances of item
item+ - One or more instances of item
item ~ n - Exactly n instances of item
item ~ n..m - Between n to m instances of item (not recommended for wide ranges, due to performance issues)


In [10]:
oct = """set maxdepth 100

10 * { ry 36 sat 0.9 } 30 * { ry 10 } 1 * { h 30 b 0.8 sat 0.8 a 0.3  } r1

rule r1 w 20 {
   { s 0.9 rz 5 h 5 rx 5 x 1 }  r1
   { s 1 0.2 0.5 } box
}

rule r1 w 20 {
   {  s 0.99 rz -5 h 5 rx -5 x 1 }   r1
   { s 1 0.2 0.5 } box
}

rule r1  {
}"""

print(Itest().convert(parser.parse(oct)))

len(call kids) 7
ry 36 
ry 10
   
len(call kids) 2
sa 0.9 rz 5  rx 5 tx 1
len(call kids) 2
s 1 0.2 0.5
len(call kids) 2
sa 0.99 rz -5  rx -5 tx 1
len(call kids) 2
s 1 0.2 0.5
<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call count="1" transforms="" rule="r1"/>
  </rule>
  <rule name="r1" weight="20">
    <call transforms="sa 0.9 rz 5  rx 5 tx 1" rule="r1"/>
    <instance transforms="s 1 0.2 0.5" shape="box"/>
  </rule>
  <rule name="r1" weight="20">
    <call transforms="sa 0.99 rz -5  rx -5 tx 1" rule="r1"/>
    <instance transforms="s 1 0.2 0.5" shape="box"/>
  </rule>
  <rule name="r1"/>
</rules>



In [11]:
oct2 = """set maxdepth 100

10 * { x 1} 20 * { y 2 }  r1

rule r1  {
    box
}"""

t = parser.parse(oct2)
print(Itest().convert(t))

"""
<?xml version="1.0" ?>
<rules max_depth="100">
  <rule name="entry">
    <call transforms="tx 1" count="10" rule="r1_00"/>
  </rule>
  <rule name="r1_00">
    <call transforms="ty 2" count="20" rule="r1"/>
  </rule>
  <rule name="r1">
    <instance transforms="" shape="box"/>
  </rule>
</rules>
"""

len(call kids) 5
tx 1
ty 2
len(call kids) 1
<?xml version="1.0" ?>
<rules>
  <rule name="entry">
    <call count="20" transforms="ty 2" rule="r1"/>
  </rule>
  <rule name="r1">
    <instance shape="box"/>
  </rule>
</rules>



'\n<?xml version="1.0" ?>\n<rules max_depth="100">\n  <rule name="entry">\n    <call transforms="tx 1" count="10" rule="r1_00"/>\n  </rule>\n  <rule name="r1_00">\n    <call transforms="ty 2" count="20" rule="r1"/>\n  </rule>\n  <rule name="r1">\n    <instance transforms="" shape="box"/>\n  </rule>\n</rules>\n'

In [12]:
print(t.pretty())

main
  global_md	100
  entry
    call
      repeat	10
      loop
        trans
          x
          1
      repeat	20
      loop
        trans
          y
          2
      rulename	r1
  ruledef
    rulename	r1
    call
      rulename	box



Logic for two loops:  
if 2 loops on call:  
    make another rule from 2nd loop  
    and call it from 1st loop  

logic for more loops  
    make a rule for each extra loop  
    call them in  a chain  

can we push the repeat down to the loop level like this

```
main
  global_md	100
  entry
    call     
      loop
        repeat	10
        trans
          x
          1     
      loop
        repeat	20
        trans
          y
          2
      rulename	r1
  ruledef
    rulename	r1
    call
      rulename	box
```
This would mean we had all the bits for a new rule in one place.

In [13]:
print(t.pretty())

main
  global_md	100
  entry
    call
      repeat	10
      loop
        trans
          x
          1
      repeat	20
      loop
        trans
          y
          2
      rulename	r1
  ruledef
    rulename	r1
    call
      rulename	box

